# Test for CCS-AMP Implementations

This file seeks to test my implementation against the implementation by Vamsi.

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import FactorGraphGeneration as FGG

# OuterCode1 = FGG.Triadic8(15)
OuterCode1 = FGG.Triadic10(12)
# OuterCode1 = FGG.Triadic12(10)
# OuterCode1 = FGG.Triadic15(8)


## Fast Hadamard Transforms

The ```PyFHT_local``` code can all be found in `pyfht`, which uses a C extension to speed up the fht function.
Only one import suffices, with the latter being much faster.

In [ ]:
# import PyFHT_local
from pyfht import block_sub_fht

## SPARC Codebook

We use the `block_sub_fht` which computes the equivalent of $A.\beta$ by using $L$ separate $M\times M$ Hadamard matrices. However we want each entry to be divided by $\sqrt{n}$ to get the right variance, and we need to do a reshape on the output to get column vectors, so we'll wrap those operations here.

Returns two functions `Ab` and `Az` which compute $A\cdot B$ and $z^T\cdot A$ respectively.

In [ ]:
def sparc_codebook(L, M, n,P):
    Ax, Ay, _ = block_sub_fht(n, M, L, seed=None, ordering=None) # seed must be explicit
    def Ab(b):
        return Ax(b).reshape(-1, 1)/ np.sqrt(n)
    def Az(z):
        return Ay(z).reshape(-1, 1)/ np.sqrt(n) 
    return Ab, Az

# Vector Approximation

This function outputs the closest approximation to the input vector given that its L1 norm is 1 and no entry is greater than 1/K

In [ ]:
def approximateVector(x, K):    

    # normalize initial value of x
    xOrig = x / np.linalg.norm(x, ord=1)
    
    # create vector to hold best approximation of x
    xHt = xOrig.copy()
    u = np.zeros(len(xHt))
    
    # run approximation algorithm
    while np.amax(xHt) > (1/K):
        minIndices = np.argmin([(1/K)*np.ones(xHt.shape), xHt], axis=0)
        xHt = np.min([(1/K)*np.ones(xHt.shape), xHt], axis=0)
        
        deficit = 1 - np.linalg.norm(xHt, ord=1)
        
        if deficit > 0:
            mIxHtNorm = np.linalg.norm((xHt*minIndices), ord=1)
            scaleFactor = (deficit + mIxHtNorm) / mIxHtNorm
            xHt = scaleFactor*(minIndices*xHt) + (1/K)*(np.ones(xHt.shape) - minIndices)

    # return admissible approximation of x
    return xHt

## Posterior Mean Estimator (PME)

This function implements the posterior mean estimator for situations where prior probabilities are uninformative.

In [ ]:
def pme0(q, r, d, tau):
    """Posterior mean estimator (PME)
    
    Args:
        q (float): Prior probability
        r (float): Effective observation
        d (float): Signal amplitude
        tau (float): Standard deviation of noise
    Returns:
        sHat (float): Probability s is one
    
    """
    sHat = ( q*np.exp( -(r-d)**2 / (2*(tau**2)) ) \
            / ( q*np.exp( -(r-d)**2 / (2*(tau**2))) + (1-q)*np.exp( -r**2 / (2*(tau**2))) ) ).astype(float)
    return sHat

# Dynamic Denoiser

This function performs believe propagation (BP) on the factor graph of the outer code.

In [ ]:
def dynamicDenoiser(r,OuterCode,K,tau,d,numBPiter):
    """
    Args:
        r (float): Effective observation
        d (float): Signal amplitude
        tau (float): Standard deviation of noise
    """
    M = OuterCode.getsparseseclength()
    L = OuterCode.getvarcount()

    p0 = 1-(1-1/M)**K
    p1 = p0*np.ones(r.shape,dtype=float)
    mu = np.zeros(r.shape,dtype=float)

    # Compute local estimate (lambda) based on effective observation using PME.
    localEstimates = pme0(p0, r, d, tau)
    
    # Reshape local estimate (lambda) into an LxM matrix
    Beta = localEstimates.reshape(L,-1)
    OuterCode.reset()
    for varnodeid in OuterCode.getvarlist():
        i = varnodeid - 1
        Beta[i,:] = approximateVector(Beta[i,:], K)
        OuterCode.setobservation(varnodeid, Beta[i,:]) # CHECK
    
    for iter in range(1):    # CHECK: Leave at 1 for now
        OuterCode.updatechecks()
        OuterCode.updatevars()

    for varnodeid in OuterCode.getvarlist():
        i = varnodeid - 1
        Beta[i,:] = OuterCode.getextrinsicestimate(varnodeid)
        mu[i*M:(i+1)*M] = approximateVector(Beta[i,:], K).reshape(-1,1)

    return mu

## AMP
This is the actual AMP algorithm. It's a mostly straightforward transcription from the relevant equations, but note we use `longdouble` types because the expentials are often too big to fit into a normal `double`.

In [ ]:
def amp_state_update(z, s, d, Ab, Az, K, numBPiter, OuterCode):

    """
    Args:
        s: State update through AMP composite iteration
        z: Residual update through AMP composite iteration
        tau (float): Standard deviation of noise
        mu: Product of messages from adjoining factors
    """
    n = z.size

    # Compute tau online using the residual
    tau = np.sqrt(np.sum(z**2)/n)

    # Compute effective observation
    r = (d*s + Az(z))

    # Compute updated state
    mu = dynamicDenoiser(r,OuterCode,K,tau,d,numBPiter)
    s = pme0(mu, r, d, tau)
        
    return s

In [ ]:
def amp_residual(y, z, s, d, Ab):
    """
    Args:
        s1: State update through AMP composite iteration
        s2: State update through AMP composite iteration
        y: Original observation
        tau (float): Standard deviation of noise
    """
    n = y.size
    
    # Compute tau online using the residual
    tau = np.sqrt(np.sum(z**2)/n)

    # Compute residual
    Onsager = d*(np.sum(s) - np.sum(s**2))   
    z_plus = y - d*Ab(s)+ (z/(n*tau**2))*(Onsager)
    
    return z_plus

# Simulation

In [ ]:
K=5 # Number of active users
B=120 # Payload size of each active user
L = OuterCode1.getvarcount() # Number of sections/sub-blocks
n=38400 # Total number of channel uses (real d.o.f)
T=12 # Number of AMP iterations
listSize = 2*K  # List size retained for each section after AMP converges
J = OuterCode1.getseclength() # Length of each coded sub-block
M = OuterCode1.getsparseseclength() # Length of each section

numBPiter = 1; # Number of BP iterations on outer code. 1 seems to be good enough & AMP theory including state evolution valid only for one BP iteration
EbNodB = 2.5 # Energy per bit. With iterative extension, operating EbN0 falls to 2.05 dB for 25 users with 1 round SIC
simCount = 2 # number of simulations

# EbN0 in linear scale
EbNo = 10**(EbNodB/10)
P = 2*B*EbNo/n
σ_n = 1
#Generate the power allocation and set of tau coefficients

# We assume equal power allocation for all the sections. Code has to be modified a little to accomodate non-uniform power allocations
Phat = n*P/L
d = np.sqrt(n*P/L)

msgDetected=0

for simIndex in range(simCount):
    print('Simulation Number: ' + str(simIndex))
    
    # Generate active users message sequences
    messages = np.random.randint(2, size=(K,B))

    # Outer-encode the message sequences
    codewords = OuterCode1.encodemessages(messages)
#     for codeword in codewords:
#         OuterCode1.testvalid(codeword)

    # Convert indices to sparse representation
    # sTrue: True state
    sTrue = np.sum(codewords,axis=0).reshape(-1,1)

    # Generate the binned SPARC codebook
    Ab, Az = sparc_codebook(L, M, n, P)
    
    # Generate our transmitted signal X
    x = np.sqrt(Phat)*Ab(sTrue)
    
    # Generate random channel noise and thus also received signal y
    noise = np.random.randn(n, 1) * σ_n
    y = (x + noise).reshape(-1, 1)

    # Run AMP decoding
    z = y.copy()
    s = np.zeros((L*M, 1))

    for t in range(T):
        s = amp_state_update(z, s, d, Ab, Az, K, numBPiter, OuterCode1)
        z = amp_residual(y, z, s, d, Ab)
    
    # Decoding wiht Graph
    originallist = codewords.copy()
    recoveredcodewords = FGG.decoder(OuterCode1,s,listSize)

    # Calculation of per-user prob err
    simMsgDetected = 0
    matches = FGG.numbermatches(originallist,recoveredcodewords)
    
    print('Matches ' + str(matches) + ' out of ' + str(K))
    msgDetected = msgDetected + matches
    
errorRate = (K*simCount - msgDetected)/(K*simCount)

print("Per user probability of error = ", errorRate)
